# Assignment 4 - TDT4265
#### Sara L. Ludvigsen and Emma H. Buøen
##### April 2020

## Task 1
### 1.a
The intersection over union (IoU), is the ratio between the intersected area over the joined area for two regions.
![IoU.png](IoU.png)

### 1.b
Precison measures how accurate your predictions is. Recall measures how good you find all the positives. When we say "true positive" we mean the cases where we predicted "positive" to be "true", and the prediction was correct. "False positive" is when we predict "positive", but it is incorrect. 

$$
TP = True Positive
$$
$$
TN = True Negative
$$
$$
FP = False Positive
$$
$$
FN = False Negative
$$

$$
Precision = \frac{TP}{TP + FP}
$$

$$
Recall = \frac{TP}{TP + FN}
$$

### 1.c

Average precision (AP) is equal to the area under the precision and recall curve. The mean average precision (mAP) is the mean of the curves. 

*Insert drawing of precision and recall curve*

$$
AP_{class1} = 0.4*1 + 0.3*0.5 + 0.3*0.2 + 0.3*0.5*0.5 + 0.3*0.3*0.5 = 0.73
$$

$$
AP_{class2} = 0.3*1 + 0.2*0.6 + 0.2*0.5 + 0.3*0.2 + 0.2*0.4*0.5 + 0.2*0.1*0.5 + 0.3*0.3*0.5 = 0.675
$$

$$
mAP = \frac{AP_{class1} + AP_{class2}}{2} = \frac{0.73 + 0.675}{2} = 0.7025
$$

## Task 3
### 3.a
*The SSD architecture produces a fixed-size number of bounding boxes and a score for each bounding box. After this, the final step is to filter out overlapping boxes, what is this called?*

The final step to filter out overlapping boxes is called a non-maximum suppression step.

### 3.b
*The SSD architecture predicts bounding boxes at multiple scales to enable the network to detect objects of different sizes. Is the following true or false: Predictions from the deeper layers in SSD are responsible to detect small objects.*

The statement 

> Predictions from the deeper layers in SSD are responsible to detect small objects.

is false. The deeper layers are reponsible to detect larger objects. As stated in the [blog post](https://medium.com/@jonathan_hui/ssd-object-detection-single-shot-multibox-detector-for-real-time-processing-9bd8deac0e06) about SSDs 

> Higher resolution feature maps are responsible for detecting small objects

and the resoultion is highest at the beginning of the network, and is reduced for each layer.

### 3.c

*SSD use k number of bounding boxes with different aspect ratios at each spatial location in a feature map to predict c class scores and 4 offsets relative to the original box shape. Why do they use different bounding box aspect ratios at the same spatial location?Why do they use different bounding box aspect ratios at the same spatial location?*

They use diffrent aspect ratios for bounding boxes at the same spatial locations so they get a diverse set of predictions, because it covers various input object shapes and sizes. They also observed that more default boxes are better. When they use 6 boxes per location, the performance is 0.6% better than with 4 boxes. 4 boxes is 2.1% better than 2 boxes. It seems that using a variety of default box shapes makes the prediction easier for the network.

### 3.d
*What is the main difference between SSD and YOLOv1/v2 (The YOLO version they refer to in the SSD paper)?*

The main difference is that the SSD model adds several feature layers to the end of the network. These layers predict the offsets and confidences to default boxes of different scales and aspect ratios. Below is an image that illustrates the different network architectures.

![Graphics/task3d_comparision.png](Graphics/task3d_comparision.png)

The image is taken from [**SSD: Single Shot MultiBox Detector** by Wei Liu, Dragomir Anguelov, Dumitru Erhan, Christian Szegedy, Scott Reed, Cheng-Yang Fu, Alexander C. Berg](https://arxiv.org/abs/1512.02325).
